# 用于处理多模态RAG图文问答挑战赛训练集JSON文件，提取问题与答案构建QA对
本notebook将演示如何将原始训练集转换为标准QA格式，便于后续微调。

In [1]:
#

In [ ]:
import json
import os
import pandas as pd

def is_answer_consistent(question, answer):
    """校验同一问题答案的一致性（示例逻辑，需根据实际场景完善）"""
    # 实际场景可通过哈希、语义相似度等方式实现
    return True  # 此处为示例，默认通过校验

def build_finance_sft_data(raw_data):
    """构建符合金融任务规范的SFT数据"""
    sft_data = []
    for item in raw_data:
        # 1. 明确金融助手角色定位
        system_prompt = "你是专业金融分析师助手，需基于研报、财报等金融文档内容回答，严格区分事实与观点，遵循金融术语规范"

        # 2. 标注任务类型（信息提取/逻辑分析/术语解读等）
        task_type = "金融问答"  # 可根据实际数据字段细化
        if "提取" in item.get("question", ""):
            task_type = "信息提取"
        elif "分析" in item.get("question", ""):
            task_type = "逻辑分析"
        elif "解释" in item.get("question", ""):
            task_type = "术语解读"

        # 3. 答案冲突校验
        if not is_answer_consistent(item.get("question", ""), item.get("answer", "")):
            continue

        sft_data.append({
            "system": system_prompt,
            "instruction": item.get("question", ""),
            "output": item.get("answer", ""),
            "task_type": task_type,
            "metadata": {  # 补充金融场景元数据
                "file_name": item.get("file_name", "未知"),
                "page": item.get("page", "未知")
            }
        })
    return sft_data

def main():
    # 1. 加载原始JSON数据集
    data_path = "spark_multi_rag/datas/多模态RAG图文问答挑战赛训练集.json"
    with open(data_path, "r", encoding="utf-8") as f:
        raw_data = json.load(f)
    print(f"原始样本数量: {len(raw_data)}")
    print("示例原始样本:", raw_data[0])

    # 2. 构建金融SFT数据
    finance_sft_data = build_finance_sft_data(raw_data)
    print(f"处理后SFT样本数量: {len(finance_sft_data)}")
    print("示例SFT样本:", finance_sft_data[0] if finance_sft_data else "无有效样本")

    # 3. 保存处理后的数据
    sft_json_path = "spark_multi_rag/datas/finance_sft_train.json"
    with open(sft_json_path, "w", encoding="utf-8") as f:
        json.dump(finance_sft_data, f, ensure_ascii=False, indent=2)
    print(f"已保存金融SFT JSON文件: {sft_json_path}")

    # 4. 保存为CSV格式（便于分析）
    sft_df = pd.DataFrame(finance_sft_data)
    sft_csv_path = "spark_multi_rag/datas/finance_sft_train.csv"
    sft_df.to_csv(sft_csv_path, index=False, encoding="utf-8")
    print(f"已保存金融SFT CSV文件: {sft_csv_path}")

if __name__ == "__main__":
    main()

## 2. 加载JSON数据集
读取多模态RAG图文问答挑战赛训练集.json文件，解析为Python对象。

## 4. 构建QA对列表
遍历数据集，将每条数据的question和answer字段提取出来，构建QA对列表。指令字段统一设置为“你是一名专业的财报数据问答助手”。

## 5. 保存处理后的QA数据
将处理后的QA对列表保存为新的JSON或CSV文件，便于后续模型训练或分析。